In [173]:
import random
import gym
import gymnasium
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Dropout
from collections import deque

In [174]:
env=gymnasium.make('MountainCar-v0',render_mode='human')

In [175]:
env.reset().reshape()

AttributeError: 'tuple' object has no attribute 'reshape'

In [160]:
env.render()

In [161]:
env.observation_space.shape[0]

2

In [162]:
env.action_space.n


np.int64(3)

In [163]:
env.action_space.sample()

np.int64(0)

In [164]:
class DQN:
    def __init__(self,env):
        self.env=env
        self.memory=deque(maxlen=2000)
        self.gamma=0.03
        self.epsilon=1.0
        self.epsilon_min=0.01
        self.epsilon_decay=0.995
        self.learning_rate=0.005
        self.tau=0.125
        self.model=self.create_model()
        self.target_model=self.create_model()
    def create_model(self):
            model=Sequential()
            model.add(Dense(24,input_dim=self.env.observation_space.shape[0],activation='relu'))
            model.add(Dense(48,activation='relu'))
            model.add(Dense(24,activation='relu'))
            model.add(Dense(self.env.action_space.n))
            model.compile(loss='mean_squared_error', optimizer=Adam(lr=self.learning_rate))
            return model
    def act(self,state):
            self.epsilon*=self.epsilon_dacay
            self.epsilon=max(self.epsilon_min, self.epsilon)
            if np.random.random()<self.epsilon:
                return  self.env.action_space.sample()
            else:
                return np.argmax(self.model.predict(state))
    def replay(self,state):
            batch_size=32
            if len(self.memory)<batch_size:
                return
            else:
                samples=random.sample(self.memory,batch_size)
                for sample in samples:
                    state,action,reward,new_state,done=sample
                    target=self.target_model.predict(state)
                    if done:
                        target[0][action]=reward
                    else:
                        Q_future=max(self.target_model.predict(new_state)[0])
                        target[0][action]=reward+Q_future*self.gamma
                    self.model.fit(state,target,epoch=1,verbose=1)
    def remember(self,state,action,reward,new_state,done):
        self.memory.append([state,action,reward,new_state,done])
    def target_train(self):
            weights=self.model.get_weights()
            target_weights=self.target_model.get_weights()
            for i in range(len(target_weights)):
                target_weights[i]=weights[i]*self.tau+target_weights[i]*(1-self.tau)
            self.target_model.set_weights(target_weights)
    def save_model(self,fn):
         self.model.save(fn)



In [165]:
env=gymnasium.make('MountainCar-v0',render_mode='human')
gamma=0.9
epsilon=0.95

In [166]:
trails=1000
trails_len=500
dpn_agent=DQN(env=env)

ValueError: Argument(s) not recognized: {'lr': 0.005}

In [153]:
steps=[]
for trail in range(trails):
    current_state=env.reset().reshape(1,2)
    for step in range(trails_len):
        print('#',step)
        action=dpn_agent.act(current_state)
        new_state,reward,done,_=env.step(action)
        new_state=new_state.reshape(1,2)
        dqn_agent.remember(current_state,action,reward,new_state,done)
        dqn_agnet.replay(current_state)
        dqn_agnet.target_train()
        current_state=new_state
        if done:
            break
        if steps>=199:
            print('faild')
        else:
            print('success')
            dqn_agnet.save_model('parnia')
            break

AttributeError: 'tuple' object has no attribute 'reshape'